In [1]:
# System imports
from datetime import datetime
import numpy as np
import pandas as pd

# Third-party imports
import yfinance as yf
from pathlib import Path

# Local imports
from portfolio.holdings import Portfolio
from portfolio.commsec import Trades
import datehandler

# Vars
today = datetime.today().date()

In [8]:
trades_df

,Ticker,Market,TradeType,Volume,TradePrice,EffectivePrice,Brokerage
Date,,,,,,,
2020-01-16,IBXO,ASX,B,100099.0,0.010000,0.010100,10.0
2020-01-14,DEM,ASX,B,4170.0,0.240000,0.242398,10.0
2020-01-14,IFN,ASX,S,1767.0,0.680000,0.668710,20.0
2020-01-10,AVH,ASX,S,1551.0,0.645000,0.638556,10.0
2020-01-10,VRS,ASX,S,4553.0,0.064000,0.061803,10.0
...,...,...,...,...,...,...,...
2017-08-03,NAN,ASX,B,2000.0,2.310700,2.320675,20.0
2016-07-06,QBE,ASX,B,500.0,10.280000,10.319900,20.0
2016-04-28,RMD,ASX,B,300.0,7.500000,7.566500,20.0


In [2]:
trades = Trades()
trades_df = trades.all

p_dates = datehandler.date_list(trades_df.index[-1], today)  # Portfolio dates index

tickers = list(sorted(set(trades_df.Ticker.to_list())))
props = ['TradeType','TradeVolume','TradePrice','HoldingVolume','Vwap']

columns = pd.MultiIndex.from_product([tickers,props], names=['Tickers','Props'])
df_p = pd.DataFrame(None, index=pd.DatetimeIndex(p_dates), columns=columns)  # Create multiindex portfolio df

# Clean index and columns
df_p.index.name = 'Date'
df_p = df_p.reindex(sorted(df_p.columns),axis=1)  # Sort columns
df_p = df_p.reindex(sorted(df_p.index),axis=0)  # Sort index
df_p

Tickers              A2M                                                 ACW  \
Props      HoldingVolume TradePrice TradeType TradeVolume Vwap HoldingVolume   
Date                                                                           
2015-01-07           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-08           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-09           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-12           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-13           NaN        NaN       NaN         NaN  NaN           NaN   
...                  ...        ...       ...         ...  ...           ...   
2020-03-23           NaN        NaN       NaN         NaN  NaN           NaN   
2020-03-24           NaN        NaN       NaN         NaN  NaN           NaN   
2020-03-25           NaN        NaN       NaN         NaN  NaN           NaN   
2020-03-26           NaN        NaN       NaN         NaN  NaN           NaN   
2020-03-27           NaN        NaN       NaN         NaN  NaN           NaN   

Tickers                                           ...           WZR  \
Props      TradePrice TradeType TradeVolume Vwap  ... HoldingVolume   
Date                                              ...                 
2015-01-07        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-08        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-09        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-12        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-13        NaN       NaN         NaN  NaN  ...           NaN   
...               ...       ...         ...  ...  ...           ...   
2020-03-23        NaN       NaN         NaN  NaN  ...           NaN   
2020-03-24        NaN       NaN         NaN  NaN  ...           NaN   
2020-03-25        NaN       NaN         NaN  NaN  ...           NaN   
2020-03-26        NaN       NaN         NaN  NaN  ...           NaN   
2020-03-27        NaN       NaN         NaN  NaN  ...           NaN   

Tickers                                                    ZNO             \
Props      TradePrice TradeType TradeVolume Vwap HoldingVolume TradePrice   
Date                                                                        
2015-01-07        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-08        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-09        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-12        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-13        NaN       NaN         NaN  NaN           NaN        NaN   
...               ...       ...         ...  ...           ...        ...   
2020-03-23        NaN       NaN         NaN  NaN           NaN        NaN   
2020-03-24        NaN       NaN         NaN  NaN           NaN        NaN   
2020-03-25        NaN       NaN         NaN  NaN           NaN        NaN   
2020-03-26        NaN       NaN         NaN  NaN           NaN        NaN   
2020-03-27        NaN       NaN         NaN  NaN           NaN        NaN   

Tickers                                
Props      TradeType TradeVolume Vwap  
Date                                   
2015-01-07       NaN         NaN  NaN  
2015-01-08       NaN         NaN  NaN  
2015-01-09       NaN         NaN  NaN  
2015-01-12       NaN         NaN  NaN  
2015-01-13       NaN         NaN  NaN  
...              ...         ...  ...  
2020-03-23       NaN         NaN  NaN  
2020-03-24       NaN         NaN  NaN  
2020-03-25       NaN         NaN  NaN  
2020-03-26       NaN         NaN  NaN  
2020-03-27       NaN         NaN  NaN  

[1363 rows x 175 columns]

In [3]:
dropped_cols = ['Market','Ticker','TradePrice','Brokerage']
for ticker in tickers:
    # Add trade dataframe values to portfolio dataframe
    df_t = trades_df[trades_df.Ticker == ticker].copy()
    df_t.loc[df_t['TradeType'] == 'S', 'Volume'] *= -1  # Don't worry about the warning
    df_t = df_t.drop(columns=dropped_cols)  # Drop columns first
    df_t = df_t.rename(columns={            # Then rename columns - due to conflicting name!
        'Volume':'TradeVolume',
        'EffectivePrice':'TradePrice',
        })
    df_p[ticker] = df_t

In [4]:
# Lookup prices
# Build list of tickers for yfinance
lookup_tickers = [f'{ticker}.AX' for ticker in df_p.columns.levels[0].to_list()]
lookup_tickers = ' '.join(lookup_tickers)

prices = yf.download(lookup_tickers, start=p_dates[0], end=p_dates[-1], group_by='ticker')

[*********************100%***********************]  35 of 35 completed

2 Failed downloads:
- SRS.AX: No data found, symbol may be delisted
- IBXO.AX: No data found for this date range, symbol may be delisted


In [5]:
tickers_with_prices = []

# Remove market suffix from data
tickers_without_suffix = [ticker.replace('.AX','') for ticker in prices.columns.levels[0]]

prices.columns.set_levels(tickers_without_suffix, level=0, inplace=True)

In [7]:
def _build_from_trades(df):
    df_ticker = df.copy()

    df_ticker['HoldingVolume'] = df_ticker['TradeVolume'].fillna(0).cumsum()

    # Vwap: BookValue / HoldingVolume
    df_ticker.loc[df_ticker['TradeType'] == 'B','TradeEncoding'] = 1
    df_ticker.loc[df_ticker['TradeType'] == 'S','TradeEncoding'] = 0
    df_ticker['BuyValue'] = df_ticker['TradeVolume'] * df_ticker['TradePrice'] * df_ticker['TradeEncoding']
    df_ticker['BuyVolume'] = df_ticker['TradeVolume'] * df_ticker['TradeEncoding']
    df_ticker['BuyVolume'] = df_ticker['BuyVolume'].cumsum()  # Temp calc column
    df_ticker['BuyValue'] = df_ticker['BuyValue'].cumsum()  # Temp calc column
    df_ticker['Vwap'] = df_ticker['BuyValue'].divide(df_ticker['BuyVolume']).fillna(method='ffill')

    df_ticker = df_ticker.drop(columns=['BuyVolume','BuyValue','TradeEncoding'])  # Remove temp calc columns
    
    return df_ticker

def _stocksplits(df):
    try:  # Dividends and stock splits
        stock = yf.Ticker(f'{ticker}.AX')
        df_actions = stock.actions
    except TypeError:
        print(f'\rSomething went wrong!',flush=True)
        return
    
    # Add stocksplits column
    if len(df_actions[df_actions['Stock Splits'] > 0]) > 0:
        df['StockSplits'] = df_actions['Stock Splits'].replace(0,np.nan)

        # Set all dates with 0 holdings to have np.nan holdings
        df['HoldingVolume'] = df['HoldingVolume'].replace(0,np.nan)

        stocksplit_cumulative = df.loc[df['HoldingVolume'].first_valid_index():]['StockSplits'].cumprod()
        stocksplit_cumulative = stocksplit_cumulative.fillna(method='ffill').fillna(1)


        # Update holding volume
        df['HoldingVolume'] *= stocksplit_cumulative
        df['HoldingVolume'] = np.ceil(df['HoldingVolume'])
        df['HoldingVolume'] = df['HoldingVolume'].fillna(0)  # Replace np.nan holdings with 0
        
        # Update vwap
        df['Vwap'] /= stocksplit_cumulative

    return df

# -------Code ---------
# Build portfolio
for count, ticker in enumerate(df_p.columns.levels[0]):
    print(f'\r{ticker} | Progress {count+1}/{len(df_p.columns.levels[0])} ',end='',flush=True)
    
    # ==== Test
    if ticker != 'RMD':  # For testing one ticker only
        continue
    # ==== Test

    if ticker not in tickers_without_suffix:
        print(f'Skipping {ticker} due to no price data...')
        continue
    
    df_t = df_p[ticker]

    df_t = _build_from_trades(df_t)     # Build porfolio from trades
    df_t['ClosePrice'] = prices[ticker]['Close'].copy()  # Load prices into dataframe
    df_t = _stocksplits(df_t)  # Update dataframe with stocksplits
    ####################################
    #### Dividends to be added manually  
    ####################################

    # Calculate cashflow
    df_t['Cashflow'] = df_t['TradePrice'] * df_t['TradeVolume']
    df_t['Cashflow'] = df_t['Cashflow'].fillna(0)
    
    # Calculate end value
    df_t['EndValue'] = df_t['HoldingVolume'] * df_t['ClosePrice'].fillna(method='ffill')
    # Any trades have the end-value reset
    df_t.loc[df_t.TradeVolume > 0,'EndValue'] = df_t.loc[df_t.TradeVolume > 0]['Cashflow']
    # Add trade value to previous days close

    # Calculate start value
    df_t['StartValue'] = df_t['EndValue'].shift(1).replace(0,np.nan)

    # Calculate daily returns
    df_t['DailyReturn'] = df_t['EndValue'] / (df_t['StartValue'] + df_t['Cashflow']) - 1
    df_t['DailyReturn'] = df_t['DailyReturn'].replace(-1,np.nan) + 1  # Catch sales
    df_t['CumulativeReturn'] = df_t['DailyReturn'].cumprod() - 1

    # Drop group and re-append new df
    df_p = df_p.drop(ticker, axis='columns', level=0)
    for col in df_t.columns:
        df_p[ticker,col] = df_t[col]

    df_p[ticker].to_csv('test.csv')

# df_p[('portfolio', 'EndValue')] = df_p.xs('EndValue', level='Props', axis=1).sum(axis=1, min_count=1)
# df_p[('portfolio', 'StartValue')] = df_p.xs('StartValue', level='Props', axis=1).sum(axis=1, min_count=1)
# df_p[('portfolio', 'Cashflow')] = df_p.xs('Cashflow', level='Props', axis=1).sum(axis=1, min_count=1)
# df_p[('portfolio','DailyReturn')] = df_p[('portfolio','EndValue')]/(df_p[('portfolio','StartValue')]+df_p[('portfolio','Cashflow')]) - 1
# df_p['portfolio', 'DailyReturn'] = df_p['portfolio', 'DailyReturn'].replace(-1,np.nan) + 1
# df_p[('portfolio','CumulativeReturn')] = df_p[('portfolio','DailyReturn')].cumprod()

# Clean columns
# df_p = df_p.drop(['EndValue','StartValue','Cashflow'], axis='columns',level=1)

df_p['RMD'].loc['2016-04-27':]

ZNO | Progress 35/35

Props,HoldingVolume,TradePrice,TradeType,TradeVolume,Vwap,ClosePrice,StockSplits,Cashflow,EndValue,StartValue,DailyReturn,CumulativeReturn
Date,,,,,,,,,,,,
2016-04-27,0.0,NaN,NaN,NaN,NaN,7.590000,NaN,0.00,0.000000,NaN,NaN,NaN
2016-04-28,300.0,7.5665,B,300,7.566500,7.550000,NaN,2269.95,2269.950000,NaN,NaN,NaN
2016-04-29,300.0,NaN,NaN,NaN,7.566500,7.460000,NaN,0.00,2238.000011,2269.950000,0.985925,-0.014075
2016-05-02,300.0,NaN,NaN,NaN,7.566500,7.370000,NaN,0.00,2210.999966,2238.000011,0.987936,-0.025970
2016-05-03,300.0,NaN,NaN,NaN,7.566500,7.460000,NaN,0.00,2238.000011,2210.999966,1.012212,-0.014075
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-23,213.0,NaN,NaN,NaN,10.330585,20.040001,NaN,0.00,4268.520195,4737.119951,0.901079,1.681322
2020-03-24,213.0,NaN,NaN,NaN,10.330585,22.010000,NaN,0.00,4688.130049,4268.520195,1.098303,1.944905
2020-03-25,213.0,NaN,NaN,NaN,10.330585,22.280001,NaN,0.00,4745.640146,4688.130049,1.012267,1.981031


In [15]:
df_p['RMD'].loc[df_p['RMD'].TradeVolume.isna() == False]

Props,HoldingVolume,TradePrice,TradeType,TradeVolume,Vwap,ClosePrice,Dividends,StockSplits,Cashflow,EndValue,StartValue,DailyReturn,CumulativeReturn
Date,,,,,,,,,,,,,
2016-04-28,300.0,7.5665,B,300,7.566500,7.55,NaN,NaN,2269.95,2269.950000,NaN,NaN,NaN
2018-05-15,227.0,13.423,S,-73,7.566500,13.55,NaN,NaN,-979.88,3075.850043,4070.999908,0.99506,0.784572
2018-06-21,0.0,14.5221,S,-227,7.566500,14.58,NaN,NaN,-3296.52,0.000000,3280.149957,NaN,NaN
2018-07-03,213.0,14.2237,B,213,10.330585,14.00,NaN,NaN,3029.64,3029.640000,NaN,NaN,NaN
